In [1]:
import math

def init_computer(code, inputs):
    return {
        'mem': code.copy(),
        'mem_size': len(code),
        'extend_mem' : {},
        'inst': 0,
        'rel': 0,
        'inputs': inputs.copy(),
        'outputs': [],
        'halt': False,
        'needs_input': False
    }

def read_mem(computer, pos):
    if(pos >= computer['mem_size']):
        if(pos in computer['extend_mem']):
            return computer['extend_mem'][pos]
        else:
            return 0
    else:
        return computer['mem'][pos]

def write_mem(computer, pos, val):
    if(pos < 0):
        print("invalid mem pos %i" % pos)
        return
    if(pos >= computer['mem_size']):
        computer['extend_mem'][pos] = val
    else:
        computer['mem'][pos] = val


def run(computer):
    code_size = len(computer['mem'])
    i = computer['inst']
    outputs = []
    op_info = {1:3, 2:3, 3:1, 4:1, 5:2, 6:2, 7:3, 8:3, 9:1, 99:0}
    computer['needs_input'] = False
    while(True):
        op = read_mem(computer, i)
        opcode = op % 100
        if(not(opcode in op_info)):
            print("error unknown opcode %i" % (opcode))
            computer['needs_input'] = False
            break
        a0 = -1
        a1 = -1
        a2 = -1
        jump = False
        if(op_info[opcode] > 0):
            p_mode = (math.floor(op / 100) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a0 = read_mem(computer, i + 1)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a0 = i + 1
            elif( p_mode == 2 ):
                #relative mode
                a0 = read_mem(computer, i + 1) + computer['rel']
        if(op_info[opcode] > 1):
            p_mode = (math.floor(op / 1000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a1 = read_mem(computer, i + 2)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a1 = i + 2
            elif( p_mode == 2 ):
                #relative mode
                a1 = read_mem(computer, i + 2) + computer['rel']
        if(op_info[opcode] > 2):
            p_mode = (math.floor(op / 10000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a2 = read_mem(computer, i + 3)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a2 = i + 3
            elif( p_mode == 2 ):
                #relative mode
                a2 = read_mem(computer, i + 3) + computer['rel']
        if(opcode == 1):
            #add op
            write_mem(computer, a2, read_mem(computer, a0) + read_mem(computer, a1))
        elif(opcode == 2):
            #mult op
            write_mem(computer, a2, read_mem(computer, a0) * read_mem(computer, a1))
        elif(opcode == 3):
            #read op
            if(len(computer['inputs']) == 0):
                computer['needs_input'] = True
                break
            write_mem(computer, a0, computer['inputs'][0])
            computer['inputs'] = computer['inputs'][1:]
        elif(opcode == 4):
            outputs.append(read_mem(computer, a0))
        elif(opcode == 5):
            #jump if true op
            if(read_mem(computer, a0) != 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 6):
            #jump if false op
            if(read_mem(computer, a0) == 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 7):
            #check less than op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) < read_mem(computer, a1)) else 0)
        elif(opcode == 8):
            #check equals op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) == read_mem(computer, a1)) else 0)
        elif(opcode == 9):
            #change relative param op
            computer['rel'] = computer['rel'] + read_mem(computer, a0)
        elif(opcode == 99):
            #halt op
            computer['halt'] = True
            computer['needs_input'] = False
            break
        if(not(jump)):
            i = i + op_info[opcode] + 1
        if(i >= code_size):
            print('exiting b/c end of code reached')
            computer['needs_input'] = False
    computer['outputs'] = outputs
    computer['inst'] = i
    
    return computer


In [13]:
import random

class vec2i(object):
    def __init__(self, x, y):
        self.x = int(round(x))
        self.y = int(round(y))
    
    #@classmethod
    #def from_vec2f(cls, vf):
        #return cls(int(round(vf.x)), int(round(vf.y)))

    @classmethod
    def from_index(cls, ind):
        ssplit = ind[1:].split('P')
        yoff = 1
        if(len(ssplit) == 1):
            ssplit = ind[1:].split('N')
            yoff = -1
        
        xoff = 1
        if(ind[0] == 'N'):
            xoff = -1
        
        return cls(xoff * int(ssplit[0]), yoff * int(ssplit[1]))
        
    def index(self):
        xpart = ""
        if(self.x < 0):
            xpart = "N%i" % abs(self.x) 
        else:
            xpart = "P%i" % abs(self.x)
        ypart = ""
        if(self.y < 0):
            ypart = "N%i" % abs(self.y) 
        else:
            ypart = "P%i" % abs(self.y)
        return "%s%s" % (xpart,ypart)
    
    def step(self, v2):
        return vec2i(self.x + v2.x, self.y + v2.y)
    
    def diff(self, v2):
        return vec2i(v2.x - self.x, v2.y - self.y)
    
    def copy(self):
        return vec2i(self.x, self.y)
    
    def __str__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
    def __repr__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
class Area(object):
    def __init__(self):
        self.map = {}
        self.leftmost = 0
        self.upmost = 0
        self.rightmost = 0
        self.downmost = 0
        self.move_steps = {1:vec2i(0,-1), 2:vec2i(0,1), 3:vec2i(-1,0), 4:vec2i(1,0)}
        self.parent = {}
        self.parent_move = {}
        self.open = set()
        self.closed = set()
        self.mapscore = {}
        self.finaldest = ''
        self.floodval = {}
        self.floodopen = set()
        self.floodclosed = set()
        
        startind = vec2i(0,0).index()
        self.closed.add(startind)
        self.parent[startind] = 'ROOT'
        self.mapscore[startind] = 0
        for idir in range(1,5):
            dirind = self.move_steps[idir].index()
            self.open.add(dirind)
            self.parent[dirind] = startind
            self.mapscore[dirind] = 1
            self.parent_move[dirind] = self.rev_dir(idir)
        
    def rev_dir(self, forward):
        if(forward == 1):
            return 2
        elif(forward == 2):
            return 1
        elif(forward == 3):
            return 4
        elif(forward == 4):
            return 3
        
    def wall(self, v):
        key = v.index()
        if(not(key in self.map)):
            return 0
        return self.map[key]
    
    def wallind(self, key):
        if(not(key in self.map)):
            return 0
        return self.map[key]
    
    def path_to_root(self, v):
        cursor = v
        path = [cursor]
        while(cursor != 'ROOT'):
            if(not(cursor in self.parent)):
                print('couldn\'t find parent for %s' % cursor)
                return None
            cursor = self.parent[cursor]
            path.append(cursor)
        return path
    
    def moves_between(self, v1, v2):
        path1 = self.path_to_root(v1)
        path2 = self.path_to_root(v2)
        
        smaller = len(path1)
        if(len(path2) < smaller):
            smaller = len(path2)
        i = -1
        while(True):
            if(i < -smaller):
                break
            if(path1[i] != path2[i]):
                break
            i = i -1
        path2_part = path2[:i+1]
        path2_part.reverse()
        return [self.parent_move[p1] for p1 in path1[:i+1]] + [self.rev_dir(self.parent_move[p2]) for p2 in path2_part]
    
    def set_map(self, v, kind):
        key = v.index()
        
        if(not(key in self.closed)):
            self.map[key] = kind
            if(kind == 3):
                self.finaldest = key
            if(key in self.open):
                self.open.remove(key)
                self.closed.add(key)
            if(v.x < self.leftmost):
                self.leftmost = v.x
            if(v.y < self.upmost):
                self.upmost = v.y
            if(v.x > self.rightmost):
                self.rightmost = v.x
            if(v.y > self.downmost):
                self.downmost = v.y

    def move(self, v1, v2, kind):
        from_key = v1.index()
        to_key = v2.index()
        
        for idir in range(1,5):
            s = v2.step(self.move_steps[idir]).index()
            if(not(s in self.closed) and not(s in self.open)):
                self.open.add(s)
                self.parent[s] = to_key
                self.mapscore[s] = self.mapscore[to_key] + 1
                self.parent_move[s] = self.rev_dir(idir)
            
        if(not(to_key in self.closed)):
            self.set_map(v2, kind)
            
    def render(self, v = None):
        width = self.rightmost - self.leftmost + 1
        height = self.downmost - self.upmost + 1
        image = ''
        for y in range(height):
            for x in range(width):
                cursor = vec2i(x + self.leftmost, y + self.upmost)
                mapind = self.wall(cursor) % 10
                if(v!= None and cursor.x == v.x and cursor.y == v.y):
                    image = image + 'R'
                elif(mapind == 0): #unknown
                    image = image + ' '
                elif(mapind == 1): #floor
                    image = image + '.'
                elif(mapind == 2): #wall
                    image = image + '#'
                elif(mapind == 3): #oxygen_system
                    image = image + 'O'
            image = image + '\n'
        return image
    
    def flood(self):
        if(self.finaldest == ''):
            print("can't flood, haven't found final dest!")
            return
        
        self.floodval[self.finaldest] = 0
        self.floodclosed.add(self.finaldest)
        
        v = vec2i.from_index(self.finaldest)
        openlist = []
        opencount = 0
        for i in range(1, 5):
            s = v.step(self.move_steps[i]).index()
            kind = self.wallind(s)
            if(kind == 1): #floor
                self.floodopen.add(s)
                openlist.append(s)
                opencount = opencount + 1
                self.floodval[s] = 1
        
        highestspilltime = 0
        
        while(opencount > 0):
            nexts = openlist[0]
            openlist = openlist[1:]
            opencount = opencount - 1
            self.floodopen.remove(nexts)
            self.floodclosed.add(nexts)
            v = vec2i.from_index(nexts)
            spilltime = self.floodval[nexts] + 1
            
            for i in range(1, 5):
                s = v.step(self.move_steps[i]).index()
                if((s in self.floodclosed) or (s in self.floodopen)):
                    continue
                    
                kind = self.wallind(s)
                if(kind == 1): #floor
                    self.floodopen.add(s)
                    openlist.append(s)
                    opencount = opencount + 1
                    self.floodval[s] = spilltime
                    if(spilltime > highestspilltime):
                        highestspilltime = spilltime
        
        return highestspilltime
            
            
                

class Droid(object):
    def __init__(self, code, area):
        self.computer = init_computer(code, [])
        self.area = area
        self.pos = vec2i(0,0)
        self.move_steps = {1:vec2i(0,-1), 2:vec2i(0,1), 3:vec2i(-1,0), 4:vec2i(1,0)}
        self.dir_priority = [1,3,2,4]
        self.randstrats = {0:[1,1,1,1,1,1,1,1,1,1],
                           1:[2,2,2,2,2,2,2,2,2,2,2],
                           2:[3,3,3,3,3,3,3,3],
                           3:[4,4,4,4,4,4,4,4,4],
                           4:[1,2,2,3,3,3,4,4,4,4,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4]}
        
    def exec1(self):
        inputs = self.computer['inputs'].copy()
        self.computer = run(self.computer)
        outputs = self.computer['outputs']
        self.computer['outputs'] = []
        
        if(len(inputs) != len(outputs)):
            print('different number of inputs(%i) and outputs(%i)' % (len(inputs), len(outputs)))
            return
        
        for i in range(len(inputs)):
            stepin = inputs[i]
            stepout = outputs[i]
            
            if(stepout == 0): #hit wall, didn't move
                self.area.set_map(self.pos.step(self.move_steps[stepin]),2)
            elif(stepout == 1): #moved onto floor
                pos_to = self.pos.step(self.move_steps[stepin])
                self.area.move(self.pos, pos_to, 1)
                self.pos = pos_to
            elif(stepout == 2): #moved into oxygen system
                pos_to = self.pos.step(self.move_steps[stepin])
                self.area.move(self.pos, pos_to, 3)
                self.pos = pos_to
            else:
                print('unknown output %i' % stepout)
    
    def execmult(self, ins):
        self.computer['inputs'] = ins
        self.exec1()
        
    def execrand(self):
        strat = random.randint(0, 9)
        if(strat < 5):
            self.execmult(self.randstrats[strat].copy())
        else:
            numsteps = random.randint(0,12)
            steps = []
            for i in range(numsteps):
                steps.append(random.randint(1,4))
            self.execmult(steps)
            
    def autopilot(self):
        if(self.computer['halt']):
            print("can't autopilot, computer has shut down")
            return
        opens = list(self.area.open)
        if(len(opens) == 0):
            print('the world is smaller than it used to be')
        else:
            dest = opens[random.randint(0,len(opens)-1)]
            moves = self.area.moves_between(self.pos.index(), dest)
            #print('from %s => %s' % (self.pos.index(), dest))
            #print("moves: ", end ="")
            #print(moves)
            self.execmult(moves)
    
    def render(self):
        print(self.area.render(self.pos))
        print(self.area.render())



In [14]:
area = Area()
code = [3,1033,1008,1033,1,1032,1005,1032,31,1008,1033,2,1032,1005,1032,58,1008,1033,3,1032,1005,1032,81,1008,1033,4,1032,1005,1032,104,99,1001,1034,0,1039,1002,1036,1,1041,1001,1035,-1,1040,1008,1038,0,1043,102,-1,1043,1032,1,1037,1032,1042,1105,1,124,1001,1034,0,1039,1001,1036,0,1041,1001,1035,1,1040,1008,1038,0,1043,1,1037,1038,1042,1106,0,124,1001,1034,-1,1039,1008,1036,0,1041,101,0,1035,1040,1001,1038,0,1043,102,1,1037,1042,1105,1,124,1001,1034,1,1039,1008,1036,0,1041,1002,1035,1,1040,101,0,1038,1043,101,0,1037,1042,1006,1039,217,1006,1040,217,1008,1039,40,1032,1005,1032,217,1008,1040,40,1032,1005,1032,217,1008,1039,5,1032,1006,1032,165,1008,1040,35,1032,1006,1032,165,1102,1,2,1044,1106,0,224,2,1041,1043,1032,1006,1032,179,1102,1,1,1044,1105,1,224,1,1041,1043,1032,1006,1032,217,1,1042,1043,1032,1001,1032,-1,1032,1002,1032,39,1032,1,1032,1039,1032,101,-1,1032,1032,101,252,1032,211,1007,0,44,1044,1105,1,224,1102,0,1,1044,1106,0,224,1006,1044,247,1002,1039,1,1034,1001,1040,0,1035,1001,1041,0,1036,1002,1043,1,1038,102,1,1042,1037,4,1044,1105,1,0,5,26,24,17,68,40,71,9,36,46,67,39,48,8,20,23,12,47,28,13,47,2,68,17,71,31,63,31,83,14,78,31,8,33,30,63,30,5,7,11,91,97,17,84,23,37,46,6,14,59,1,76,41,63,85,83,86,63,33,13,50,17,37,16,59,8,7,35,71,9,23,67,46,62,58,38,76,3,71,43,17,64,29,30,72,91,17,70,21,15,76,31,89,20,38,27,65,53,60,34,90,99,56,15,45,57,8,52,70,36,15,79,32,35,83,78,10,3,90,16,74,14,84,43,20,81,91,25,71,83,24,31,92,72,34,59,27,78,6,31,14,31,76,9,80,63,35,40,92,12,84,65,41,27,82,10,7,56,25,70,4,98,16,37,65,46,78,11,97,20,16,95,98,24,31,3,57,74,42,99,36,34,74,10,81,46,43,97,2,24,61,55,13,96,41,41,46,14,64,2,46,94,53,3,3,81,37,85,7,54,29,90,22,75,47,20,26,86,69,53,89,17,2,55,13,85,99,90,2,48,29,66,55,31,19,39,59,56,98,28,38,10,46,10,62,20,63,18,53,97,9,32,6,46,3,91,24,6,62,30,73,26,24,50,3,16,78,3,34,50,8,18,40,65,64,21,28,30,87,45,99,8,21,77,40,73,38,56,12,86,64,43,61,89,4,55,47,28,14,8,99,52,51,40,82,26,19,68,17,53,70,5,14,22,64,69,84,14,69,2,80,18,79,5,66,18,34,48,31,34,54,50,8,33,73,38,52,94,71,7,31,94,31,93,66,82,39,40,42,80,91,70,10,6,50,35,96,13,7,89,22,58,30,24,85,81,88,55,7,58,38,91,55,11,35,84,28,87,26,78,48,66,11,88,8,18,68,55,38,6,1,57,60,1,8,99,58,21,29,88,32,32,57,72,8,20,45,5,91,39,51,59,82,29,52,37,33,49,5,28,38,17,6,58,67,11,72,51,42,4,3,12,94,84,25,31,72,32,89,49,4,23,57,49,27,38,50,30,23,15,80,4,12,67,14,48,76,91,58,11,63,37,95,1,15,22,84,8,23,87,61,32,78,87,7,47,1,81,31,84,91,21,19,68,6,87,3,72,43,60,23,67,42,40,62,9,86,33,84,69,24,97,37,49,24,67,2,16,52,3,42,49,3,95,84,61,8,40,79,10,74,51,6,77,63,1,66,7,55,24,80,68,17,30,47,54,30,77,40,99,18,85,99,85,2,27,18,33,54,99,27,5,64,39,22,66,12,71,29,26,35,49,13,41,22,76,30,70,30,75,34,7,5,62,1,23,61,43,90,24,91,40,42,75,48,40,91,39,46,38,56,17,28,51,56,7,51,40,56,22,87,43,99,6,58,93,35,47,83,10,57,55,68,34,68,93,28,55,11,3,53,80,9,41,42,50,95,7,4,84,10,91,33,12,99,98,60,76,73,24,70,46,72,27,36,62,27,25,43,59,39,9,95,72,9,17,79,36,52,52,22,4,55,57,16,19,65,62,83,11,76,73,37,89,21,86,6,88,17,93,1,59,8,48,73,90,96,10,85,46,12,99,16,16,76,4,2,2,45,62,30,12,14,72,60,9,19,71,43,41,36,99,69,38,1,1,48,32,33,83,26,15,51,19,31,71,92,8,49,34,87,32,80,73,28,65,95,7,8,85,12,63,22,83,8,70,1,82,96,59,29,95,43,59,72,68,38,48,11,87,54,90,11,93,30,63,12,96,41,64,21,89,24,94,73,79,18,55,40,95,0,0,21,21,1,10,1,0,0,0,0,0,0]
droid = Droid(code, area)
droid.exec1()
print ('blarg')
#print (droid.computer)]
#print (droid.computer)
for i in range(10000):
    droid.autopilot()
    if(len(area.open) == 0):
        print('world explored in %i iterations' % i)
        break

droid.render()
print('part 1 answer: %i' % (len(area.path_to_root(area.finaldest)) - 2))
print('part 2 answer: %i' % area.flood())

blarg
world explored in 1658 iterations
 ######### ####### ################# ### 
#.........#R......#.................#...#
#.#.#####.#####.#.#.#############.#.#.#.#
#.#.....#.......#.#...#...#...#...#.#.#.#
#.#####.#.###########.#.#.#.#.#.## ##.#.#
#...#...#.#.........#...#...#.#...#...#.#
 ##.#.###.#.#####.###########.###.#.###.#
#.#.#.#...#.#...#.........#.#...#.#...#.#
#.#.#.#.###.###.#######.#.#.###.#.###.#.#
#.#.#.#...#.....#.....#.#.....#.#...#.#.#
#.#.#.#.#######.###.###.#.#####.#.###.#.#
#.#.#.#.#...........#...#.#.....#.....#.#
#.#.#.###.###########.#####.###########.#
#...#...........#...#.#.......#.....#...#
#.#####.#########.#.#.#.#####.#.###.#.#.#
#.#...#.#.........#.#...#...#.#.#.....#.#
 ##.#.###.#########.#####.#.###.#.#####.#
#...#...........#...#.....#...#.#.#...#.#
#.## ##########.#.###.#######.#.###.#.## 
#...#.#.......#.#.....#.......#.....#...#
 ##.#.#.#####.#########.###########.###.#
#.#...#...#...#...... #.....#.....#...#.#
#.###.###.#.###.###### ####.#.#.#.##